In [ ]:
import os, sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

path = os.getcwd()
ffpath="/home/rty10/Documents/forcefields/"

In [ ]:

FFNAME="czapla2022"


In [ ]:
theta_lst = ['tilt','roll','twist','shift','slide','rise']

lstRR = ['AG','GG','AA','GA']
lstYR = ['CG','CA','TA']
lstRY = ['AT','AC','GC']
dimer_lst = lstRY + lstRR + lstYR

PARLIMITS = {'tilt':[-2, 2],
             'roll':[-2.0, 8.0],
             'twist':[28,38],
             'shift':[-0.5,0.5], 
             'slide':[-0.75,0.75], 
             'rise':[3.1, 3.5]}

-------------
## Comparison: Olson1998 and Culled Czapla2022 Dataset

### Step Parameters

In [ ]:
compffdf = pd.read_csv("StepParameters_"+FFNAME+"_dim.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
compffdf = compffdf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})

olsondf = pd.read_csv(ffpath+"/RestStateParameters/StepParameters_Olson1998.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
olsondf = olsondf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})


In [ ]:
for theta in ['twist']:#theta_lst:
    
    testdf = pd.DataFrame(columns=['step', 'Olson1998', FFNAME], index=[i for i in range(len(dimer_lst))])
    for i in range(len(dimer_lst)):
        testdf.at[i, 'step']=dimer_lst[i]
        testdf.at[i, 'Olson1998']=olsondf.loc[olsondf.step==dimer_lst[i]][theta].item()
        testdf.at[i, FFNAME]=compffdf.loc[compffdf.step==dimer_lst[i]][theta].item()
    
testdf

In [ ]:
for theta in theta_lst:

    fig, axes = plt.subplots(1, 3, figsize=(5,2.5), sharey=True, gridspec_kw={"width_ratios":[1.5, 2, 1.5]})

    testdf = pd.DataFrame(columns=['step', 'Olson1998', FFNAME], index=[i for i in range(len(dimer_lst))])
    for i in range(len(dimer_lst)):
        testdf.at[i, 'step']      = dimer_lst[i]
        testdf.at[i, 'Olson1998'] = olsondf.loc[olsondf.step==dimer_lst[i]][theta].item()
        testdf.at[i, FFNAME]      = compffdf.loc[compffdf.step==dimer_lst[i]][theta].item()
    
    testdf.loc[testdf.step.isin(lstYR)].plot.bar(x='step', rot=0, color=['red','dodgerblue'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[0])
    axes[0].set_title("YR Steps")
    testdf.loc[testdf.step.isin(lstRR)].plot.bar(x='step', rot=0, color=['red','dodgerblue'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[1])
    axes[1].set_title("RR Steps")
    testdf.loc[testdf.step.isin(lstRY)].plot.bar(x='step', rot=0, color=['red','dodgerblue'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[2])
    axes[2].set_title("RY Steps")
    
    for ax in axes:
        ax.set_ylabel(theta.capitalize() )
        ax.set_xlabel('')
        ax.set_ylim(PARLIMITS[theta][0], PARLIMITS[theta][1])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
    handles, labels = axes[0].get_legend_handles_labels()
    
    lgd = fig.legend(handles, labels, loc="lower center", ncol=2, bbox_to_anchor=(0.5, -0.1))
    
    plt.tight_layout()
    #plt.savefig("czapla22_olson1998-comparison_"+theta+"_v03.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.clf()
    del handles, labels
    del testdf

### Force Constant Comparison

In [ ]:
compfcdf = pd.read_csv("ForceConstants_"+FFNAME+"_dim.txt", 
                        header=None, index_col=0, sep=',\s+|={', engine="python")
compfcdf[36]=compfcdf[36].map(lambda x: x.rstrip('},'))
compfcdf=compfcdf.astype(float)

olsonfcdf = pd.read_csv(ffpath+"/ForceConstants/ForceConstants_Olson1998.txt", 
                        header=None, index_col=0, sep=',\s+|={', engine="python")
olsonfcdf[36]=olsonfcdf[36].map(lambda x: x.rstrip('},'))
olsonfcdf=olsonfcdf.astype(float)

In [ ]:
olson_dim_df  = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
olson_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in olsonfcdf.index:
    
    mat    = np.reshape(olsonfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        olson_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            olson_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            olson_data_df.at[b1, b2] = np.sqrt( olson_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            olson_data_df.at[b1, b2] = 0
del olson_dim_df
olson_data_df = round(olson_data_df.astype(float), 1)
olson_data_df

In [ ]:
compeg_dim_df = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
compff_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in compfcdf.index:
    
    mat    = np.reshape(compfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        compeg_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            compeg_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            compff_data_df.at[b1, b2] = np.sqrt( compeg_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            compff_data_df.at[b1, b2] = 0
del compeg_dim_df
compff_data_df = round(compff_data_df.astype(float), 1)
compff_data_df

In [ ]:
DATAMIN = 0.0 #np.sqrt( olson_dim_df['eigenvalue_product'].min() )
DATAMAX = 10.0 #np.sqrt( young_dim_df['eigenvalue_product'].max() )

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(8,5))

sns.heatmap(data=compff_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( compff_data_df==0 ),
            ax=ax2)

plt.yticks(rotation=0)
ax2.set_title(FFNAME+"\n")
ax2.xaxis.tick_top()

#plt.tight_layout()
#plt.show()
#plt.savefig("young2019-dim_heatmap_eigenvalue-analysis_v01.png", dpi=300)
#plt.clf()

#fig, ax = plt.subplots(figsize=(4,4))
sns.heatmap(data=olson_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( olson_data_df==0 ),
            ax=ax1)
ax1.set_title("Olson1998\n")
ax1.xaxis.tick_top()
plt.yticks(rotation=0)


plt.tight_layout()
plt.savefig("czapla22_olson1998-eigenvalue-comparison_v03.png", dpi=300)
plt.show()
plt.clf()

del DATAMIN, DATAMAX

-------------------

## Olson1998 to Czapla2022 at 3.5-Ang Cutoff

In [ ]:
olsondf = pd.read_csv(ffpath+"/RestStateParameters/StepParameters_Olson1998.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
olsondf = olsondf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})

olsonfcdf = pd.read_csv(ffpath+"/ForceConstants/ForceConstants_Olson1998.txt",  header=None, index_col=0, sep=',\s+|={', engine="python")
olsonfcdf[36]=olsonfcdf[36].map(lambda x: x.rstrip('},'))
olsonfcdf=olsonfcdf.astype(float)



compffdf = pd.read_csv("pdb_res-series-data/StepParameters_czapla2022_res035_dim.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
compffdf = compffdf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})

compfcdf = pd.read_csv("pdb_res-series-data/ForceConstants_czapla2022_res035_dim.txt", header=None, index_col=0, sep=',\s+|={', engine="python")
compfcdf[36]=compfcdf[36].map(lambda x: x.rstrip('},'))
compfcdf=compfcdf.astype(float)



### Step Parameters

In [ ]:
for theta in theta_lst:

    fig, axes = plt.subplots(1, 3, figsize=(5,2.5), sharey=True, gridspec_kw={"width_ratios":[1.5, 2, 1.5]})

    testdf = pd.DataFrame(columns=['step', 'Olson1998', FFNAME], index=[i for i in range(len(dimer_lst))])
    for i in range(len(dimer_lst)):
        testdf.at[i, 'step']      = dimer_lst[i]
        testdf.at[i, 'Olson1998'] = olsondf.loc[olsondf.step==dimer_lst[i]][theta].item()
        testdf.at[i, FFNAME]      = compffdf.loc[compffdf.step==dimer_lst[i]][theta].item()
    
    testdf.loc[testdf.step.isin(lstYR)].plot.bar(x='step', rot=0, color=['red','green'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[0])
    axes[0].set_title("YR Steps")
    testdf.loc[testdf.step.isin(lstRR)].plot.bar(x='step', rot=0, color=['red','green'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[1])
    axes[1].set_title("RR Steps")
    testdf.loc[testdf.step.isin(lstRY)].plot.bar(x='step', rot=0, color=['red','green'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[2])
    axes[2].set_title("RY Steps")
    
    for ax in axes:
        ax.set_ylabel(theta.capitalize() )
        ax.set_xlabel('')
        ax.set_ylim(PARLIMITS[theta][0], PARLIMITS[theta][1])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
    handles, labels = axes[0].get_legend_handles_labels()
    
    lgd = fig.legend(handles, ["Olson1998", "Czapla2022, 3.5$\AA$"], loc="lower center", ncol=2, bbox_to_anchor=(0.5, -0.1))
    
    plt.tight_layout()
    plt.savefig("czapla2022-res035_olson1998-comparison_"+theta+"_v01.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.clf()
    del handles, labels
    del testdf

### Force Constants

In [ ]:
olson_dim_df  = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
olson_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in olsonfcdf.index:
    
    mat    = np.reshape(olsonfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        olson_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            olson_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            olson_data_df.at[b1, b2] = np.sqrt( olson_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            olson_data_df.at[b1, b2] = 0
del olson_dim_df
olson_data_df = round(olson_data_df.astype(float), 1)


compeg_dim_df = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
compff_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in compfcdf.index:
    
    mat    = np.reshape(compfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        compeg_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            compeg_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            compff_data_df.at[b1, b2] = np.sqrt( compeg_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            compff_data_df.at[b1, b2] = 0
del compeg_dim_df
compff_data_df = round(compff_data_df.astype(float), 1)


In [ ]:
DATAMIN = 0.0 #np.sqrt( olson_dim_df['eigenvalue_product'].min() )
DATAMAX = 10.0 #np.sqrt( young_dim_df['eigenvalue_product'].max() )

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(8,5))

sns.heatmap(data=compff_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( compff_data_df==0 ),
            ax=ax2)

plt.yticks(rotation=0)
ax2.set_title(FFNAME+",  3.5$\AA$\n")
ax2.xaxis.tick_top()

#plt.tight_layout()
#plt.show()
#plt.savefig("young2019-dim_heatmap_eigenvalue-analysis_v01.png", dpi=300)
#plt.clf()

#fig, ax = plt.subplots(figsize=(4,4))
sns.heatmap(data=olson_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( olson_data_df==0 ),
            ax=ax1)
ax1.set_title("Olson1998\n")
ax1.xaxis.tick_top()
plt.yticks(rotation=0)


plt.tight_layout()
plt.savefig("czapla2022-res035_olson1998-eigenvalue-comparison_v01.png", dpi=300)
plt.show()
plt.clf()

del DATAMIN, DATAMAX

## 2.0-Ang 

In [ ]:
olsondf = pd.read_csv(ffpath+"/RestStateParameters/StepParameters_Olson1998.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
olsondf = olsondf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})

olsonfcdf = pd.read_csv(ffpath+"/ForceConstants/ForceConstants_Olson1998.txt",  header=None, index_col=0, sep=',\s+|={', engine="python")
olsonfcdf[36]=olsonfcdf[36].map(lambda x: x.rstrip('},'))
olsonfcdf=olsonfcdf.astype(float)



compffdf = pd.read_csv("pdb_res-series-data/StepParameters_czapla2022_res020_dim.txt", header=None, index_col=None, sep="={|}|, ", engine="python")
compffdf = compffdf.drop([7], axis=1).rename(columns={0:'step',1:'tilt',2:'roll',3:'twist',4:'shift',5:'slide',6:'rise'})

compfcdf = pd.read_csv("pdb_res-series-data/ForceConstants_czapla2022_res020_dim.txt", header=None, index_col=0, sep=',\s+|={', engine="python")
compfcdf[36]=compfcdf[36].map(lambda x: x.rstrip('},'))
compfcdf=compfcdf.astype(float)



### Step Parameters

In [ ]:
for theta in theta_lst:

    fig, axes = plt.subplots(1, 3, figsize=(5,2.5), sharey=True, gridspec_kw={"width_ratios":[1.5, 2, 1.5]})

    testdf = pd.DataFrame(columns=['step', 'Olson1998', FFNAME], index=[i for i in range(len(dimer_lst))])
    for i in range(len(dimer_lst)):
        testdf.at[i, 'step']      = dimer_lst[i]
        testdf.at[i, 'Olson1998'] = olsondf.loc[olsondf.step==dimer_lst[i]][theta].item()
        testdf.at[i, FFNAME]      = compffdf.loc[compffdf.step==dimer_lst[i]][theta].item()
    
    testdf.loc[testdf.step.isin(lstYR)].plot.bar(x='step', rot=0, color=['red','violet'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[0])
    axes[0].set_title("YR Steps")
    testdf.loc[testdf.step.isin(lstRR)].plot.bar(x='step', rot=0, color=['red','violet'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[1])
    axes[1].set_title("RR Steps")
    testdf.loc[testdf.step.isin(lstRY)].plot.bar(x='step', rot=0, color=['red','violet'], edgecolor='black', width=0.75, alpha=0.75, legend=False, ax=axes[2])
    axes[2].set_title("RY Steps")
    
    for ax in axes:
        ax.set_ylabel(theta.capitalize() )
        ax.set_xlabel('')
        ax.set_ylim(PARLIMITS[theta][0], PARLIMITS[theta][1])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
    handles, labels = axes[0].get_legend_handles_labels()
    
    lgd = fig.legend(handles, ["Olson1998", "Czapla2022, 2.0$\AA$"], loc="lower center", ncol=2, bbox_to_anchor=(0.5, -0.1))
    
    plt.tight_layout()
    plt.savefig("czapla2022-res020_olson1998-comparison_"+theta+"_v01.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.clf()
    del handles, labels
    del testdf

### Force Constants

In [ ]:
olson_dim_df  = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
olson_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in olsonfcdf.index:
    
    mat    = np.reshape(olsonfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        olson_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            olson_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            olson_data_df.at[b1, b2] = np.sqrt( olson_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            olson_data_df.at[b1, b2] = 0
del olson_dim_df
olson_data_df = round(olson_data_df.astype(float), 1)


compeg_dim_df = pd.DataFrame(index=dimer_lst, columns=["eigenvalue_product"]+theta_lst)
compff_data_df = pd.DataFrame(index=['A','G','C','T'],
                             columns=['A','G','C','T'])

for STEP in compfcdf.index:
    
    mat    = np.reshape(compfcdf.loc[STEP].to_numpy(), (6,6))
    
    eigen_mat = np.linalg.eig( mat )[0]
    eigen_inv = np.linalg.eig( np.linalg.inv(mat) )[0]
    
    if STEP in dimer_lst:
        compeg_dim_df.at[STEP, "eigenvalue_product"] = np.prod( eigen_inv )
        
        for i in range(len(theta_lst)):
            compeg_dim_df.at[STEP, theta_lst[i]] = eigen_inv[i]

    del mat, eigen_mat, eigen_inv

for b1 in ['A','G','C','T']:
    for b2 in ['A','G','C','T']:
        if b1+b2 in dimer_lst:
            compff_data_df.at[b1, b2] = np.sqrt( compeg_dim_df.at[b1+b2, 'eigenvalue_product'] )
        else:
            compff_data_df.at[b1, b2] = 0
del compeg_dim_df
compff_data_df = round(compff_data_df.astype(float), 1)


In [ ]:
DATAMIN = 0.0 #np.sqrt( olson_dim_df['eigenvalue_product'].min() )
DATAMAX = 10.0 #np.sqrt( young_dim_df['eigenvalue_product'].max() )

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(8,5))

sns.heatmap(data=compff_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( compff_data_df==0 ),
            ax=ax2)

plt.yticks(rotation=0)
ax2.set_title(FFNAME+",  2.0$\AA$\n")
ax2.xaxis.tick_top()

#plt.tight_layout()
#plt.show()
#plt.savefig("young2019-dim_heatmap_eigenvalue-analysis_v01.png", dpi=300)
#plt.clf()

#fig, ax = plt.subplots(figsize=(4,4))
sns.heatmap(data=olson_data_df,
            cbar=True, annot=True,fmt='.1f',
            cmap="YlOrBr", vmin=DATAMIN, vmax=DATAMAX,
            cbar_kws={'orientation':'horizontal','pad':0.05, 'shrink':0.50, 'label':'Step Volume (deg$^3$$\AA$$^3$)'},
            linewidths=1,linecolor='white',
            mask=( olson_data_df==0 ),
            ax=ax1)
ax1.set_title("Olson1998\n")
ax1.xaxis.tick_top()
plt.yticks(rotation=0)


plt.tight_layout()
plt.savefig("czapla2022-res020_olson1998-eigenvalue-comparison_v01.png", dpi=300)
plt.show()
plt.clf()

del DATAMIN, DATAMAX